# Initialize Code

In [2]:
# notice
# <_1 1st, _2 2nd ,,, > : tuple
# [ ... ] : list
# { keytype, valtype } : dict

In [3]:
# import request
import requests

# import pattern
from pattern import web
from pattern.web import Element
from pattern.web import plaintext

# import regular expression
import re

# import json
import json


In [3]:
def get_page(url) :
    r = requests.get(url)
    
    if r.status_code is not 200 :
        return None
    
    return r.text

# Get Movie Counts per Year

Crawl 'http://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn' to get movie list




In [4]:
def get_movie_count_on_year() :
    # result { str year, int count }
    result = dict()
    
    # crawl $url_dir
    url_dir = 'http://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'
    page = get_page(url_dir)
    
    # find data
    elem_root = Element(page)
    
    elem_old_content = elem_root('div[id="old_content"]')
    if len(elem_old_content) is not 1 :
        return None
    
    elem_td = elem_old_content[0]('td')
    if len(elem_td) is 0 :
        return None
    
    # parse
    td_list = map(lambda x : plaintext(x.content), elem_td)
    
    year_count_list = map(lambda x : re.split('\W+' ,x)[0:2], td_list)
    
    # save at result   
    for year_count in year_count_list :
        result[year_count[0]] = int(year_count[1])
    
    return result

In [5]:
movie_counts = get_movie_count_on_year()

# Get MetaData For Each Movie

Crawl http://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open=XXXX&page=XXXX to get pair of (movie name, movie code)

movie code is used to search movie info at http://movie.naver.com/movie/bi/mi/detail.nhn?code=XXXX


In [24]:
# return [ <str name, str code> ]
def get_movie_list_of_year_page(year, page) :
    url_base = 'http://movie.naver.com/movie/sdb/browsing/bmovie.nhn?'
    url_append = 'open=' + year + '&page=' + page
    
    # find data
    page = get_page(url_base + url_append)
    
    elem_root = Element(page)
    
    elem_old_content = elem_root('div[id="old_content"]')
    if len(elem_old_content) is not 1 :
        return []
    
    elem_li = elem_old_content[0]('ul[class="directory_list"] > li')
    if len(elem_li) is 0 :
        return []
    
    elem_a = map(lambda x : x('a')[0], elem_li)
    
    # parse
    hrefs = map(lambda x : x.attrs['href'], elem_a)
    codes = map(lambda x : re.split('\D+', x)[1] if len(re.split('\D+',x )) is 2 else None, hrefs)
    names = map(lambda x : x.content, elem_a)
    
    
    # save to result
    result = map(lambda x : (names[x], codes[x]), range(len(codes)))
    
    return result

def get_movie_list_of_year(year) :
    page_count = movie_counts[year] / 20
    if movie_counts[year] % 20 is not 0 :
        page_count = page_count + 1
    
    list_per_page = map(lambda x : get_movie_list_of_year_page(year, str(x)), range(1, page_count + 1))
    list_per_year = reduce(lambda a, b : a + b, list_per_page, [])
    
    
    return list_per_year

In [31]:
# filter
movie_counts['1940'] = 0
movie_counts['1950'] = 0
movie_counts['1960'] = 0
movie_counts['1970'] = 0
movie_counts['1980'] = 0
movie_counts['2017'] = 0

# save to json
for year in movie_counts :
    if movie_counts[year] is not 0 :
        tmp = get_movie_list_of_year(year)

        data_file = open('movie_list_' + year + '.txt', 'w')
        json.dump(tmp, data_file, ensure_ascii=True)
        data_file.close()


# Load MetaData

In [4]:
def load_list_of_year(year) :
    data_file = open('movie_list_' + year + '.txt', 'r')
    data = json.load(data_file)
    data_file.close()
    
    return data

In [5]:
# load list from jsons
years = ['2015', '2014', '2016', '2011', '2010', '2013', '2012', '1991', '1990', 
         '1993', '1992', '1995', '1994', '1997', '1996', '1999', '1998', '2002', 
         '2003', '2000', '2001', '2006', '2007', '2004', '2005', '2008', '2009']

In [6]:
movie_list_of_year = dict()

# TODO: iterate through years and load
movie_list_of_year[years[0]] = load_list_of_year(years[0])



# Get MovieData
data at http://movie.naver.com/movie/bi/mi/detail.nhn?code=XXXX and http://movie.naver.com/movie/bi/mi/point.nhn?code=XXXX
 




In [125]:

# { str param, object obj }
# param : 
#    name
#    watcher_rating
#    netizen_rating
#    expert_rating
#    step1 : genre, nation, runtime, open_date
#    director
#    actor
#    reviews
def get_movie_info(code) :
    # result { str param, object obj }
    result = dict()
    
    url_base = 'http://movie.naver.com/movie/bi/mi/detail.nhn?code='
    r = requests.get(url_base + str(code))
    
    elem_root = Element(r.text);
    
    elem_info_area = elem_root('div[class="mv_info_area"]')
    if len(elem_info_area) is not 1 :
        return None
    
    # name
    elem_movie_name = elem_info_area[0]('h3[class="h_movie"] > a')
    if len(elem_movie_name) is not 1 :
        return None
    
    result['name'] = plaintext(elem_movie_name[0].content)
    
    # rate
    elem_rating_group = elem_info_area[0]('div[class="main_score"]')
    if len(elem_rating_group) is 1 :
    
        # watcher rate
        elem_actual_rate = elem_rating_group[0]('a[id="actualPointPersentBasic"] > div > em')
        list_actual_rate = map(lambda x : plaintext(x.content), elem_actual_rate)
        actual_rate = reduce(lambda a, b : a + b, list_actual_rate, '')
    
        result['watcher_rating'] = actual_rate

        # netizen rate
        elem_netizen_rate = elem_rating_group[0]('a[id="pointNetizenPersentBasic"] > em')
        list_netizen_rate = map(lambda x : plaintext(x.content), elem_netizen_rate)
        netizen_rate = reduce(lambda a, b : a + b, list_netizen_rate, '')
    
        result['netizen_rate'] = netizen_rate

        #expert rate
        elem_expert_rate = elem_rating_group[0]('div[class="spc_score_area"] > a > div > em')
        list_expert_rate = map(lambda x : plaintext(x.content), elem_expert_rate)
        expert_rate = reduce(lambda a, b : a + b, list_expert_rate, '')
    
        result['expert_rate'] = expert_rate

#TODO : Fix Code to Parse Easily    
    # step1 : genre, nation, runtime, open_date
    elem_steps = elem_info_area[0]('dd')
    if len(elem_steps) is not 0 :
        elem_spans = elem_steps[0]('p > span')
        
        span_list = list()
        for elem_span in elem_spans :
            span_list.append(elem_span)
        result['step1'] = span_list

        
    #    actor
    
    #    director
    
    #    reviews
    
    return result

# { str param, object obj }
# param : 
#    name : str
#    watcher_rating : float
#    netizen_rating : float
#    expert_rating : float
#    genre : [ int genre_num ]
#    nation : [ str nation_code ]
#    runtime : int minute
#    open_date : datetime
#    
def parse_movie_info(info) :
    return

In [126]:
tmp = get_movie_info(99701)

In [127]:
print tmp['step1'][1]

<a href="/movie/sdb/browsing/bmovie.nhn?nation=US">미국</a><!-- N=a:ifo.nation -->, <a href="/movie/sdb/browsing/bmovie.nhn?nation=GB">영국</a><!-- N=a:ifo.nation -->
